In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

**PranshuSama**

In [ ]:
from tqdm import tqdm
tqdm.pandas()
train_df = pd.read_csv("/kaggle/input/boundary-box-data/dataset/train.csv")

In [ ]:
train_df_updated = train_df.copy()
def update_range(row):
    xmax = row['xmax']
    xmin = row['xmin']
    ymax = row['ymax']
    ymin = row['ymin']
    row['xmax'] = xmax*2
    row['xmin'] = xmin*2
    row['ymax'] = ymax*2
    row['ymin'] = ymin*2
    xmax = row['xmax']
    xmin = row['xmin']
    ymax = row['ymax']
    ymin = row['ymin']
    if (xmax>1920):
        row['xmax'] = 1920
    if (xmin<0):
        row['xmin'] = 0
    if (ymax>1080):
        row['ymax'] = 1080
    if (ymin<0):
        row['ymin'] = 0
    row['bbox'] = [row['xmin'],row['xmax'],row['ymin'],row['ymax']]
    return row
train_df_updated = train_df_updated.progress_apply(update_range,axis=1)
train_df2 = train_df_updated.copy()
train_df2 = train_df2.groupby(['image_path','class'])['bbox'].apply(list).reset_index(name='bboxes')
train_df2    

In [ ]:
from sklearn.model_selection import train_test_split
df_train,df_valid = train_test_split(train_df2,test_size=0.1,random_state=42,shuffle=True)
df_train = df_train.reset_index(drop=True)
df_valid = df_valid.reset_index(drop=True)

In [ ]:
!mkdir /kaggle/working/Theme1_new_data/
!mkdir /kaggle/working/Theme1_new_data/images/
!mkdir /kaggle/working/Theme1_new_data/labels/
!mkdir /kaggle/working/Theme1_new_data/images/train/
!mkdir /kaggle/working/Theme1_new_data/images/validation/
!mkdir /kaggle/working/Theme1_new_data/labels/train/
!mkdir /kaggle/working/Theme1_new_data/labels/validation/
!mkdir /kaggle/working/Theme1_new_data/images/test/

In [ ]:
output_path = "/kaggle/working/Theme1_new_data"
import os
os.listdir(output_path)

In [ ]:
df_train.head()

In [ ]:
df_train.bboxes[0]

In [ ]:
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt

show_img_path = '/kaggle/input/boundary-box-data/dataset/images/'+df_train.image_path[1]

# Load the image using PIL
image = Image.open(show_img_path)

# Convert the image to a numpy array
image_array = np.array(image)

# Display the image
plt.imshow(image_array)
plt.axis('off')  # Remove the axis labels
plt.show()


In [ ]:
from PIL import Image, ImageDraw
import matplotlib.pyplot as plt



In [ ]:
from PIL import Image, ImageDraw
import matplotlib.pyplot as plt

# Load the image using PIL
show_img_path = '/kaggle/input/boundary-box-data/dataset/images/'+ df_train.image_path[0]
image = Image.open(show_img_path)

# Retrieve the bounding box coordinates from the DataFrame column
bounding_boxes = df_train.bboxes[0]  # Assuming the bounding box data is in the first row of the column

# Draw the bounding boxes on the image
draw = ImageDraw.Draw(image)

for bbox in bounding_boxes:
    x0, y0, x1, y1 = bbox  # Assuming the bounding box coordinates are in [x0, y0, x1, y1] format
    x_min, y_min = min(x0, x1), min(y0, y1)
    x_max, y_max = max(x0, x1), max(y0, y1)
    draw.rectangle([(x_min, y_min), (x_max, y_max)], outline='red', width=5)

# Display the image with the bounding boxes
plt.imshow(image)
plt.axis('off')  # Remove the axis labels
plt.show()


In [ ]:
import shutil
import numpy as np
data_path = "/kaggle/input/boundary-box-data/dataset/images"
def process_data(data,data_type="train"):
    for i,row in tqdm(data.iterrows(),total=len(data)):
        image_name = row['image_path'][:-4]
        bounding_boxes = row['bboxes']
        classs = row['class']
        yolo_data=[]
        for bbox in bounding_boxes:
            xmin = bbox[0]
            xmax = bbox[1]
            ymin = bbox[2]
            ymax = bbox[3]
            width = (xmax-xmin)
            height = (ymax-ymin)
            x_center = xmin+width/2
            y_center = ymin+height/2
            x_center /= 1920.0
            y_center /= 1080.0
            width/=1920.0
            height/=1080.0
            yolo_data.append([classs,x_center,y_center,width,height])
        yolo_data = np.array(yolo_data)
        np.savetxt(os.path.join(output_path,f"labels/{data_type}/{image_name}.txt"),yolo_data,fmt=["%d","%f","%f","%f","%f"])
        shutil.copyfile(os.path.join(data_path,f"{image_name}.jpg"),os.path.join(output_path,f"images/{data_type}/{image_name}.jpg"))


In [ ]:
process_data(df_train,data_type="train")

In [ ]:
process_data(df_valid,data_type="validation")

In [ ]:
test_df = pd.read_csv("/kaggle/input/boundary-box-data/dataset/test.csv")
test_df.head()

In [ ]:
import shutil
import numpy as np
data_path = "/kaggle/input/boundary-box-data/dataset/images"
def process_test_data(data):
    for i,row in tqdm(data.iterrows(),total=len(data)):
        image_name = row['image_path'][:-4]
        shutil.copyfile(os.path.join(data_path,f"{image_name}.jpg"),os.path.join(output_path,f"images/test/{image_name}.jpg"))
process_test_data(test_df)

In [ ]:
train_df.head(10)

**LETS VISUALIZE PITHOLES AND OBJECTS ON ROAD**

In [ ]:
train_df['bbox'] = train_df['xmax'].astype(str) + ','+ train_df['xmin'].astype(str)+','+train_df['ymax'].astype(str) +','+train_df['ymin'].astype(str)
train_df.head(10)

In [ ]:
train_df['bbox_list'] = train_df['bbox'].apply(lambda x: [x])


In [ ]:
train_df.head()

In [ ]:
train_df.bbox[0]

In [ ]:
from PIL import Image, ImageDraw
import matplotlib.pyplot as plt

# Load the image using PIL
show_img_path = train_df.image_path[0]
show_img_path = '/kaggle/input/boundary-box-data/dataset/images/'+ train_df.image_path[0]
image = Image.open(show_img_path)
image = Image.open(show_img_path)

# Retrieve the bounding box coordinates from the DataFrame column
bounding_boxes =df_train.bboxes[0] 
font_size = 1000 # Assuming the bounding box data is in the first row of the column
label = train_df.name[9]
# Draw the bounding boxes on the image
draw = ImageDraw.Draw(image)
for bbox in bounding_boxes:
    x0, y0, x1, y1 = bbox  # Assuming the bounding box coordinates are in [x0, y0, x1, y1] format
    x_min, y_min = min(x0, x1), min(y0, y1)
    x_max, y_max = max(x0, x1), max(y0, y1)
    draw.rectangle([(x_min, y_min), (x_max, y_max)], outline='red', width=5)
    draw.text((x_min, y_min), label, fill='black', font_size=font_size)


# Display the image with the bounding boxes
plt.imshow(image)
plt.axis('off')  # Remove the axis labels
plt.show()

In [ ]:
import ast

In [ ]:
train_df.bbox = train_df.bbox.apply(ast.literal_eval)

In [ ]:
smart_yaml = { "train" : '/kaggle/working/Theme1_new_data/images/train',"val" : '/kaggle/working/Theme1_new_data/images/validation',"nc":11,"names": ["GRAFFITI","FADED SIGNAGE","POTHOLES","GARBAGE","CONSTRUCTION ROAD","BROKEN_SIGNAGE","BAD STREETLIGHT","BAD BILLBOARD","SAND ON ROAD","CLUTTER_SIDEWALK","UNKEPT_FACADE"]}
smart_yaml

In [ ]:
os.chdir('/kaggle/working')
!pwd

In [ ]:
import yaml
#some_dict = {'a': 1, 'b': 2}

with open(r'smart_yaml.yaml', 'w') as file:
    doc = yaml.dump(smart_yaml,file)


In [ ]:
data = '/kaggle/working/Theme1_new_data'
data_img = '/kaggle/working/Theme1_new_data/images'
data_labels = '/kaggle/working/Theme1_new_data/labels'
data_img_train = '/kaggle/working/Theme1_new_data/images/train'
data_img_val = '/kaggle/working/Theme1_new_data/images/validation'
data_label_train = '/kaggle/working/Theme1_new_data/labels/train'
data_label_val  = '/kaggle/working/Theme1_new_data/labels/validation'

In [ ]:
train_df.head(5)

In [ ]:
!git clone https://github.com/ultralytics/yolov5

In [ ]:
%cd yolov5

In [ ]:
yaml_dir = '/kaggle/working/smart_yaml.yaml'

In [ ]:
!pip install ultralytics


In [ ]:
# with open('/kaggle/working/output_file.pkl', 'rb') as fin:
#     new_dict = pkl.load(fin)
!python /kaggle/working/yolov5/train.py --img 640 --cfg /kaggle/working/yolov5/models/yolov5s.yaml  --batch 16 --epochs 5 --data /kaggle/working/smart_yaml.yaml --weights yolov5s.pt --workers 24 


**RESULTS**

In [ ]:
import torch
os.listdir("runs/train/exp/weights")
model_weights = 'runs/train/exp/weights/best.pt'

In [ ]:
test_paths = '/kaggle/input/smartathon-theme-1-with-yolov5-new/Theme1_new_data/images/test'
# os.listdir(test_paths)
!python detect.py --weights runs/train/exp/weights/best.pt  --source /kaggle/input/smartathon-theme-1-with-yolov5-new/Theme1_new_data_FCNN/test --save-txt
results = "/kaggle/working/yolov5/runs/detect/exp/labels/"

In [ ]:
diction = {"0":"GRAFFITI","1":"FADED SIGNAGE","2":"POTHOLES","3":"GARBAGE","4":"CONSTRUCTION ROAD","5":"BROKEN_SIGNAGE","6":"BAD STREETLIGHT","7":"BAD BILLBOARD","8":"SAND ON ROAD","9":"CLUTTER_SIDEWALK","10":"UNKEPT_FACADE" }
len(os.listdir("/kaggle/working/yolov5/runs/detect/exp2/labels/"))
test_df['image_path']
ans = [["class", "image_path", "name", "xmax", "xmin", "ymax", "ymin"]]

In [ ]:
for dirname, _, filenames in os.walk("/kaggle/working/yolov5/runs/detect/exp2/labels/"):
    for filename in filenames:
        i = 0
        lst = []
        image_id = filename[:-4]+".jpg"
        with open(results+filename, 'r') as f:
            data = f.readline()
            data = data.split(" ")
            tst = (" ").join(data)
            tst = tst[:-1]
            xmax = int((float(data[1])+(float(data[3])/2))*1920)
            xmin = int((float(data[1])-(float(data[3])/2))*1920)
            ymax = int((float(data[2])+(float(data[4])/2))*1080)
            ymin = int((float(data[2])-(float(data[4])/2))*1080)
            ans.append([data[0], image_id, (diction[(tst[0])]),xmax,xmin,ymax, ymin])

In [ ]:
final = pd.DataFrame(ans[1:],columns=ans[0])
final.head()

In [ ]:
final

In [ ]:
final.to_csv('myans.csv',index =False)